In [1]:
import sys, os
sys.path.append(os.path.abspath('..'))

In [2]:
import torch
from Data import GraphDataset
from Models import GCNFeatureExtractor
from utils.train import train


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\melan\shared-folder\Thesis\SSL-for-Quantum-Cirucits-Generation\qiskit-env\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\melan\shared-folder\Thesis\SSL-for-Quantum-Cirucits-Generation\qiskit-env\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\melan\shared-folder\Thesis\SSL-fo

In [3]:
use_pre_paired = True
batch_size = 16
lr = 1e-3
epochs = 50
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embedding_size = 25

In [4]:
# load data 
from Data import load_graphs

basic_data, _ = load_graphs(subset='basic')

Loaded 3 elements from subset basic:
	Collected 1 sample from null_ops.
	Collected 4 items from commutations.
	Collected 7 items from equivalences.
Loaded 12 samples and 12 quantum circuits from subset.


In [5]:
# import pickle

# def load_graphs(data_dir='../Data/raw/', file_name='handcrafted_dataset.pkl', subset=None):
#     file_path = os.path.join(data_dir, file_name)
#     graphs, qcs = [], []
    
#     with open(file_path, 'rb') as f:
#         dataset = pickle.load(f)

#     if file_name == 'handcrafted_dataset.pkl':
#         if subset is not None:
#             dataset = dataset[subset]
#             print(f"Loaded {len(dataset)} elements from subset {subset}:")
#         # collect all graphs in dataset, which are stored in a nested dictionary
#         dataset = collect_from_dict(dataset) 

#     # extract graphs and qcs separately, if needed
#     for sample in dataset:
#         if isinstance(sample, tuple): # if a tuple (qc, graph)
#             g, qc = sample
#             graphs.append(g)
#             qcs.append(qc)
#         elif all(isinstance(graph, tuple) for graph in sample):# if a list of tuples (qc, graph) 
#             g, qc = zip(*sample)
#             qcs.append(list(qc))
#             graphs.append(list(g))

#     print(f"Loaded {len(graphs)} samples and {len(qcs)} quantum circuits from subset.")
    
#     return graphs, qcs

# def collect_from_dict(dictionary):
#     graphs = []
#     for k, v in dictionary.items():
#         if isinstance(v, dict):
#             graphs.extend(collect_from_dict(v))
#         elif isinstance(v, list):
#             if all(isinstance(item, list) for item in v):  # if list of lists
#                 graphs.extend(v)
#                 print(f"\tCollected {len(v)} items from {k}.")
#             elif all(isinstance(item, tuple) for item in v):  # if list of tuples
#                 graphs.append(v)
#                 print(f"\tCollected 1 sample from {k}.")
                
#     return graphs

## Basic transforms training

In [6]:
dataset = GraphDataset(basic_data, pre_paired=use_pre_paired)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)

In [7]:
graphs = basic_data[0]
print(len(graphs))

18


In [13]:
from networkx import attr_matrix


attr_matrix(dataset[0][0])

# TODO: you have to fix QCG, so that the attributes the graph has are exactly the one that you want
# Maybe create feature matrix before the graph, or something similar
# Indeed when you produce the dataset, you have no way to access attributes of the correspondent qcg object
# you used to create the graph...
# then use this nx.get_node_attributes(G, 'feature_vector') and when creating a node set as attribute 'feature_vector'
# the one-hot encoding you like

(array([[0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 1., 0., 0.]]),
 ['cx_1_target_7',
  'cx_1_control_5',
  'cx_0_control_7',
  'cx_0_target_5',
  'cx_1_control_6',
  'h_0_4',
  'cx_0_target_6'])

In [9]:
n_features = dataset[0][0].x.size(1) if use_pre_paired else dataset[0].x.size(1)
print(f'Number of features: {n_features}')
gnn = GCNFeatureExtractor(in_channels=n_features, out_channels=embedding_size)

model = torch.nn.Sequential(gnn).to(device)

AttributeError: 'DiGraph' object has no attribute 'x'

In [ ]:
list = []
list1= [1,2,3]
list2= [4,5,6]  

list.append(list1)
print(list)
list.append(list2)
print(list)

[[1, 2, 3]]
[[1, 2, 3], [4, 5, 6]]
